![Logo](../logo.png)

# Intermédiaire ②

Il nous reste plus que quelques choses à voir pour être opérationnel dans l'usage du langage et de tout ce qui l'entoure. Après l'étude de quelques bibliothèques standard majeures et l'installation d'une première bibliothèque tierce, vous devriez être capable de créer des projets Python de plus grande ampleur.

# Notions de ce cours

* 🖊️ Les décorateurs
* 🖊️ La levée d'exceptions
* 🧰 Bibliothèque standard - os.path
* 🧰 Bibliothèque standard - datetime
* 🧰 Bibliothèque standard - json
* ⚙️ Les environnements virtuels
* ⚙️ Installer un paquet avec PIP
* 📚 Bibliothèque tierce - Pillow

---

## 🖊️ Les décorateurs

Il existe en Python un moyen de créer et utiliser des "fonctions d'ordre supérieur" (high-order functions). Derrière ce nom étrange se cache le fait de créer une fonction qui accepte une fonction existante pour permettre plus tard de l'appeler comme d'habitude, mais en réalisant au passage certaines opérations supplémentaires. L'intérêt est donc de pouvoir rajouter à la volée un comportement sur des fonctions existantes sans avoir à modifier ces dernières.

En Python, ces fonctions d'ordre supérieur sont appelées des "décorateurs". Lorsque l'on écrit une fonction "décoratrice", elle va accepter en argument la fonction que l'on souhaite "décrorer", puis renvoie une nouvelle fonction créée à la volée qui elle appellera la fonction d'origine (celle qu'on décore), tout en exécutant un nouveau comportement juste avant ou juste après.

Ensuite, c'est lorsque l'on écrit la défintion d'une fonction que l'on peut demander à Python de la décorer, à l'aide d'un décorateur donc.

![Explication des décorateurs](../assets/decorator.png)

Étant une notion un peu complexe à concevoir, explorons-la petit à petit. Tout d'abord, écrivons une fonction décoratrice qui s'occupera juste d'afficher un message avant d'appeler la fonction que l'on souhaite décorer.

In [14]:
# On crée notre fonction décoratrice acceptant en argument la fonction à décorer
def mon_decorateur(fonction_a_decorer):
    # Au sein de la fonction, on va créer une nouvelle fonction...
    def wrapper():
        # Ici, le comportement que l'on veut ajouter
        print("Une fonction va être appeleée et ce message s'affiche avant.")
        # Puis l'on finit par exécuter la fonction décorée, tout en renvoyant son résultat
        return fonction_a_decorer()
    # Au lieu de renvoyer la fonction décorée, on renvoie justement notre nouvelle fonction
    return wrapper

Comme dans d'autres langages, on peut définir une fonction au sein d'une fonction, qui n'existera donc que dans un scope local. C'est pour cela qu'on va retourner cette fonction tout juste créée, afin qu'elle soit récupérée et appelée plus tard.

In [15]:
# La fonction que l'on voudra décorer
def bonjour():
    print("Aloha !")

bonjour()

Aloha !


Pour l'instant, on définit simplement une fonction et l'on teste son comportement en l'appelant.

In [16]:
# La déclaration originale de notre fonction
def bonjour():
    print("Aloha !")

# On écrase ici la déclaration originale de la fonction par sa version décorée
# Notez d'ailleurs que, ici, on passe la fonction en tant que variable, on ne l'appelle pas (car pas de parenthèses après son nom) 
#                         ↓
bonjour = mon_decorateur(bonjour)

# On appelle notre fonction comme si de rien n'était !
bonjour()

Une fonction va être appeleée et ce message s'affiche avant.
Aloha !


Pour décorer notre fonction, on va donc écraser sa définition originale, en la réattribuant avec ce que renvoie notre décorateur.

Lorsque l'on appelera notre fonction à l'avenir, on appellera en réalité sa version "décorée", avec le comportement supplémentaire désiré !

Voyons désormais la façon la plus pratique de décorer une fonction.

In [17]:
# On demande à Python d'utiliser le décorateur "mon_decorateur" pour la déclaration
# de fonction qui se trouvera à la ligne suivante
@mon_decorateur
def bonjour():
    print("Aloha !")

bonjour()

Une fonction va être appeleée et ce message s'affiche avant.
Aloha !


Comme vous avez pu le constater, on écrit directement le nom de notre décorateur précédé d'un arobase `@` juste au-dessus de la déclaration de notre fonction.

C'est la syntaxe propre à Python qui permet de décorer immédiatement une fonction que l'on va déclarer, c'est donc en général la façon la plus simple d'utiliser un décorateur lorsqu'il s'agit du code que l'on écrit nous-même.

---

## 🖊️ La levée d'exceptions

Après avoir vu comment attraper les exceptions qui peuvent se produire lors de l'exécution de notre code, il est temps d'apprendre à créer nos propres exceptions. Lorsqu'il arrive une erreur suffisemment importante pour vouloir la "remonter", la "propager" à travers notre code, on dit que l'on va "lever" (raise) une exception.

La façon la plus simple de lever une exception, dans le cas où quelque chose ne se passe vraiment pas comme prévu, est d'utiliser le mot-clé `assert` : on donne une expression booléenne censée être vraie lorsque tout va bien, et lorsqu'elle est fausse, Python lèvera une exception crée à la volée, de type `AssertionError` (classe hériant de `Exception`).

Son usage se fait sous la forme `assert [expression booléenne], [message de l'exception]`

In [20]:
order_card_currency = "yen"

assert order_card_currency == "euro", "On n'accepte que les paiements en euro !"

print("")

AssertionError: On n'accepte que les paiements en euro !

L'erreur affichée par Python vous montre clairement où s'est déroulée l'exception, son type `AssertionError`, puis le message que l'on voulait afficher.

C'est une manière rapide de générer des exceptions, mais souvenez-vous que l'on capture des exceptions selon leur type pour pouvoir correctement réagir selon. `assert` génère toujours le même type d'exception `AssertionError`, donc ce n'est pas très pratique pour ça.

Une solution est tout simplement de créer un type d'exception correspondant à un cas d'erreur précis, ce qui se traduit concrètment par :

* La création d'une classe vide, héritant de `Exception` et au nom explicite représentant notre cas d'erreur
* La levée d'une instance de cette exception au moment où le cas d'erreur survient 

In [21]:
# Déclaration de la classe de notre exception
class PaymentBadCurrency(Exception):
    pass

card_currency = "yen"
if card_currency != "euro":
    # Levée d'une nouvelle instance de notre exception
    raise PaymentBadCurrency("On n'accepte que les paiements en euro !")

PaymentBadCurrency: On n'accepte que les paiements en euro !

L'intérêt des exceptions est que, même levées dans une fonction qui appelle une fonction qui en appelle un autre, on pourra toujours finir par la capturer grâce au `try/except` vu dans le chapitre précédent.

Voyons un exemple avec un enchaînement de fonctions imbriquées, et une exception qui est levée au sein de l'une d'entre elles.

In [23]:
# 3ème fonction appelée
def verifier_paiement(devise):
    print("3. Vérification du paiement...")
    if devise != "euro":
        # Levée de l'exception
        raise PaymentBadCurrency("On n'accepte que les paiements en euro !")

# 2ème fonction appelée
def gerer_commande(devise):
    print("2. Gestion de la commande...")
    verifier_paiement(devise)

# 1ère fonction appelée
def lire_formulaire(devise):
    print("1. Lecture du formulaire...")
    # Avant d'appeler la 2ème fonction, on se prépare à la capture
    # d'une éventuelle exception plus loin dans le code
    try:
        gerer_commande(devise)
    except PaymentBadCurrency as err:
        print(f"Le paiement a échoué sur la devise : {err}")
        return "Code HTTP 500 (Internal Server Error)"
    print("Formulaire géré avec succès.")
    return "Code HTTP 200 (OK)"

# Remplacez "yen" par "euro" pour tester le comportement sans erreur
lire_formulaire("yen")

1. Lecture du formulaire...
2. Gestion de la commande...
3. Vérification du paiement...
Le paiement a échoué sur la devise : On n'accepte que les paiements en euro !


'Code HTTP 500 (Internal Server Error)'

---
## 🧰 Bibliothèque standard - os.path

Cette petite bibliothèque sert à réaliser de nombreuses opérations en rapport avec des chemins de fichiers ou de dossiers.

Bien que cela peut paraître trivial, souvenez-vous que les systèmes d'exploitation ont des façons différentes de gérer les répertoires, allant même jusqu'à ne pas utiliser le même séparateur des dossiers : `\` pour Windows, `/` pour Unix (et donc aussi Linux et macOS)...

Parmi toutes les fonctions de cette bibliothèque, voici les plus utiles :

* `.join()` : construit un chemin en utilisant le bon séparateur du système d'exploitation actuel
* `.exists()` : renvoie si le chemin passé est un fichier ou dossier qui existe
* `.isfile()` : renvoie si le chemin passé est un fichier
* `.isdir()` : renvoie si le chemin passé est un dossier
* `.splitext()` : à partir d'un chemin, renvoie séparément le chemin complet du fichier, puis l'extension du fichier

In [24]:
import os

testpath = os.path.join("..", "assets", "testfile.txt")
print(testpath)
print(f"Existe ? {os.path.exists(testpath)}")
print(f"Fichier ? {os.path.isfile(testpath)}")
print(f"Dossier ? {os.path.isdir(testpath)}")
filename, ext = os.path.splitext(testpath)
print(f"Le fichier {filename} a pour extension {ext}")

..\assets\testfile.txt
Existe ? True
Fichier ? True
Dossier ? False
Le fichier ..\assets\testfile a pour extension .txt


---

## 🧰 Bibliothèque standard - datetime

Depuis très longtemps, l'informatique a choisi de stocker facilement une date et une heure donnée en comptant le nombre de secondes écoulées depuis l'[époque Unix](https://fr.wikipedia.org/wiki/Heure_Unix) située au 1er janvier 1970 à minuit (UTC). Mais en plus de ne pas être une mesure très rigoureuse, le stockage d'un nombre de secondes sous-entends le fuseau horaire UTC sans réellement l'expliciter : ce qui peut porter à confusion lors de son usage dans une base de donnes ou dans une application.

Autant pour facilier la gestion des dates que pour assurer une rigueur dans leur usage, Python propose la bibliothèque standard `datetime`, avec laquelle on utilise des objets et des méthodes pour manipuler des dates et des heures.

Cette dernière propose notamment des moyens de représenter facilement soit juste une date, soit juste un horodatage, soit une date avec horodatage. Voyons tout d'abord comment avoir la date ou l'horodatage de "maintenant" :

In [26]:
from datetime import date, datetime

date_instance = date.today()
datetime_instance = datetime.now()

print(date_instance)
print(datetime_instance)

2021-04-19
2021-04-19 18:54:59.223200


Grâce à ces fonctions, vous rencontrez déjà des instances des types `date` et `datetime` sur lesquels on peut réaliser énormément d'opérations.

On peut accéder très simplement à une valeur en particulier, par exemple :

* Sur les `date` et `datetime` : `.day` pour le jour, `.month` pour le mois, `.year` pour l'année
* Sur les `time` et `datetime` : `.hour` pour les heures, `.minute` pour les minutes, `.second` pour les secondes

In [27]:
print(f"Nous sommes en l'année {date.today().year}.")
print(f"Il est {datetime.now().hour} heures et {datetime.now().minute} minutes.")

Nous sommes en l'année 2021.
Il est 19 heures et 3 minutes.


Il est également possible de créer un de ces objets en passant chaque valeur une à une, comme ci-dessous où l'on va créer une instance de chaque type existant.

In [28]:
from datetime import date, time, datetime

date_instance = date(year=2022, month=7, day=14)
time_instance = time(hour=13, minute=14, second=31)
datetime_instance = datetime(year=2020, month=1, day=31, hour=13, minute=14, second=31)

print(date_instance)
print(time_instance)
print(datetime_instance)

2022-07-14
13:14:31
2020-01-31 13:14:31


Vous avez sûrement constaté que l'affichage de ces types, une fois implicitement convertis en `str` par la fonction `print()`, est assez rudimentaire.

Il existe un moyen de choisir comment ces types seront convertis en chaîne de caractères grâce au _formatage_. En utilisant certaines _directives_ dont la liste est sur la [documentation Python](https://docs.python.org/fr/3/library/datetime.html#strftime-and-strptime-format-codes), on peut rapidement indiquer les valeurs que l'on veut afficher, et surtout comment.

Par exemple, si l'on veut afficher le nom complet ou abbrégé du jour, ou encore l'année sous le format de 2 ou 4 chiffres, on utilisera des _directives_ différentes.

Il suffit donc de passer notre "gabarit" à la méthode `.strftime()` sur l'instance d'un type proposé par la bibliothèque `datetime`, et les directives seront remplacées par les valeurs désirées.

In [55]:
from datetime import datetime

datetime_instance = datetime.now()

print(datetime_instance.strftime("%A %d %B, à %H:%M:%S"))

Wednesday 14 April, à 21:49:15


Ce _formatage_ peut également s'utiliser dans le cas où l'on veuille utiliser les types proposés par la bibliothèque `datetime` en partant de quelque chose écrit dans une simple chaîne de caractère. On utilisera alors aussi des _directives_ pour indiquer quelle valeur est écrite à quel endroit, et la fonction `datetime.strptime()` s'occupera de réaliser la conversion.

In [59]:
from datetime import datetime

created_at = "13/04/2021 13:37:10"

converted_created_at = datetime.strptime(created_at, "%d/%m/%Y %H:%M:%S")
print(converted_created_at)

2021-04-13 13:37:10


Il existe également un moyen d'additionner ou soustraire des valeurs à l'aide du type [timedelta](https://docs.python.org/fr/3/library/datetime.html#timedelta-objects), qui représente uniquement une durée. Par exemple, si l'on veut savoir quel jour il sera dans 2 semaines, ou dans 48 heures, etc. Il suffit d'instancier le type `timedelta` à l'image des autres types, puis de le soustraire ou de l'additionner à autre chose.

In [29]:
from datetime import timedelta

migration_duration = timedelta(weeks=2)
# Addition d'un datetime ave un timedelta
migration_end = date.today() + migration_duration

print(f"La migration de votre serveur sera terminée d'ici le {migration_end.strftime('%d/%m/%Y')}.")

La migration de votre serveur sera terminée d'ici le 03/05/2021.


Mais il est également possible de calculer une durée en réalisant des opérations entre dates, par exemple entre aujourd'hui et une date future : on obtient alors le nombre de jours restant avant d'y arriver.

In [30]:
from datetime import date

countdown = date(day=23, month=7, year=2021) - date.today()

print(f"Il reste {countdown.days} jours avant les jeux de Tokyo.")

Il reste 95 jours avant les jeux de Tokyo.


---

## 🧰 Bibliothèque standard - json

Né du monde JavaScript, JSON s'est plus ou moins imposé face au XML comme étant un format d'échange de données plus léger que ce dernier et surtout plus pratique à implémenter dans les langages qui souhaitent l'utiliser. Techniquement, JSON est un "sous-ensemble limité" du langage JavaScript, d'où le fait que son écriture ressemble à ce dernier, tout en ayant des restrictions (inexistence des commentaires...). De ce fait, vous pouvez écrire un objet JSON au milieu d'un code JavaScript et le manipuler directement.

Ce format est donc fréquemment utilisé pour échanger des informations de manière structurée avec des API, mais on peut également s'en servir pour stocker des données dans des fichiers sous un format lisible autant par les humains que par des applications. On parle alors de "document JSON", sans que l'extension de fichier `.json` ne soit imposée. La preuve, le notebook Jyputer que vous êtes en train de regarder est en réalité un document JSON !

Lorsque l'on travaille avec ce format dans un autre langage que JavaScript, on doit forcément convertir les données au format JSON en données lisibles par le langage que nous utilisons. L'action de conversion depuis le format JSON se dit _désérialiser_, et la conversion vers le format JSON se dit _sérialiser_.

Il y a peu de méthodes sur cette bibliothèque, mais leur orthographe peut semer la confusion :

* `.load()` déserialise du JSON en Python, à partir d'un fichier ouvert (avec `open()` par exemple)
* `.dump()` sérialise du Python en JSON, dans un fichier ouvert (avec `open()` par exemple)
* `.loads()` déserialise du JSON en Python, à partir d'une chaîne de caractères
* `.dumps()` sérialise du Python en JSON, dans une chaîne de caractères

Les méthodes au singulier s'utilisent donc avec des fichiers, quand celles au pluriel s'utilisent avec des chaînes de caractères.

Importons la bibliothèque puis effectuons une sérialisation toute simple d'une valeur Python :

In [14]:
import json

serialized = json.dumps(None)

print(serialized)
print(type(serialized))

null
<class 'str'>


On obtiens donc "null", stockée dans une chaîne de caractères, et l'on voit bien que le type de valeur en Python a été correctement "traduite" vers le type correspondant en JavaScript. `True` deviendra par exemple `true`, tout en minuscules. Pour le reste des valeurs, comme les nombres, les listes ou dictionnaires, leur écriture en JSON est assez similaire au Python.

Faisons désormais la sérialisation d'une valeur Python un peu plus complexe :

In [31]:
import json

py_list = [
    {
        "name": "john",
        "year": 2013,
        "is_debug": True
    }
]

json_serialized = json.dumps(py_list)

print(json_serialized)

[{"name": "john", "year": 2013, "is_debug": true}]


Ici, toute la liste et son contenu ont été converties en valeurs JSON, et réduites en une simple ligne (une chaîne de caractères). On peut donc ensuite très facilement la transmettre ou encore la sauvegarder.

Procédons maintenant à l'opération inverse, c'est à dire la désérialisation d'un objet JSON vers du Python.

In [37]:
import json

# Pour rappel, les trois guillemets permettent des chaînes multi-lignes
json_data = """
{
    "site_url": "www.superwebsite.dev",
    "site_port": 8080,
    "debug_mode": false
}
"""

python_data = json.loads(json_data)

print(python_data)
print(python_data["site_url"])

{'site_url': 'www.superwebsite.dev', 'site_port': 8080, 'debug_mode': False}
www.superwebsite.dev


Après la déserialisation de cet objet JSON, qui était un dictionnaire, on obtient également un dicitonnaire Python et l'on peut s'en servir directement en tant que tel. Bien sûr, toute modification réalisée en Python ne sera pas répliquée sur l'objet JSON original et vice-versa : il faudra à nouveau sérialiser ou désérialiser au moment voulu.

Pour ce qui est des méthodes au singulier `json.load()` et `json.dump()`, elles nécessitent donc d'être utilisées avec un fichier qui est ouvert. C'est ce que nous permet la fonction built-in `open()` vue au dernier chapitre !

Étant donné que l'on souhaite généralement stocker plusieurs valeurs dans un fichier, on utilisera alors un dictionnaire dans lequel on rangera toutes les valeurs souhaitées.

In [33]:
import json

path = os.path.join("..", "assets", "testdata.json")
# On déclare la variable en dehors du scope du "with" pour pouvoir y accéder
python_data = None

# Ouverture d'un fichier...
with open(path, "r", encoding="utf-8") as f:
    # Lecture et conversion du JSON en même temps
    python_data = json.load(f)

print(python_data)

for city in python_data['cities']:
    print(f"\n{city['name_latin']} a une population de {city['population']} habitants.")

{'cities': [{'name_ja': '東京都', 'name_latin': 'Tōkyō', 'population': 13960236}, {'name_ja': '大阪市', 'name_latin': 'Osaka', 'population': 2751862}, {'name_ja': '福岡市', 'name_latin': 'Fukuoka', 'population': 1603043}]}

Tōkyō a une population de 13960236 habitants.

Osaka a une population de 2751862 habitants.

Fukuoka a une population de 1603043 habitants.


Quant à la fonction `json.dump()`, on peut lui passer la valeur Python que l'on souhaite sérialiser ainsi que le fichier de destination que l'on vient d'ouvrir en mode écriture.


In [11]:
import json

path = os.path.join("..", "assets", "serialized.json")
python_data = {
    "languages": ["Python", "Java", "C#", "JavaScript", "HTML", "CSS"]
}

# Le mode "w" (write) va écrire dans le fichier en remplaçant tout son contenu
with open(path, "w", encoding="utf-8") as f:
    # Conversion et écriture du JSON en même temps
    json.dump(python_data, f)

Vous pouvez désormais ouvrir le fichier `serialized.json` du dossier `assets` pour y voir nos données converties au format JSON.

---

## ⚙️ Les environnements virtuels

Derrière ce nom se cache un moyen d'avoir  facilement une copie de votre installation de Python afin de travailler de façon "isolée". Le premier intérêt en utilisant cette installation isolée, cet "environnement virtuel" comme on dit, est le fait que les paquets que vous installerez avec PIP seront alors cantonés à cet environnement.

Ainsi, on évite toutes les contraintes liées au fait que les paquets soient installés d'habitude au niveau de tout votre système. Si vous avez un projet nécessitant le paquet Django en verison 2.0, puis un autre projet nécessitant sa version 3.0, le fait que chaque projet ait son propre environnement permettra d'y installer juste les paquets nécessaires, et dans les versions demandées pour un bon fonctionnement.

Bien que l'on ait l'habitude d'utiliser un projet tiers nommé `virtualenv` pour créer ces environnements, le module `venv` qui est désormais inclus à partir de Python 3.4 : nous utiliserons ce dernier par simplicité.



### Créer un environnement virtuel

La création d'un environnement virtuel se fait avec la commande suivante dans un terminal :

`python -m venv [nom]` 

Une fois situé dans le dossier racine de votre projet, la création d'un envionnement nommé par exemple "monenv" se fera alors comme ceci :

`python -m venv monenv` 

![Arborescence d'un projet avec venv](../assets/venv.png)

Comme indiqué sur l'image ci-dessus, votre environnement prend simplement la forme d'un dossier. C'est là-dedans que sont stockées les informations relatives à votre envionnement, les paquets qui y seront installés, etc.

Pour information, ce dossier n'est pas à inclure dans un dépôt Git : il faut laisser les personnes (ou les scripts automatisés) qui accèdent au projet le soin de créer eux-mêmes un environnement virtuel.


### Utiliser un environnement virtuel

Pour utiliser un environnement virtuel, on dit que l'on va activer ce dernier. Toujours dans un terminal, exécuter un certain fichier fera en sorte que l'on soit désormais "dans" l'environnement virtuel lorsque l'on fera appel à `python` ou à `pip`. Le fichier d'activation n'est pas le même selon l'invite de commande que vous utilisez :


* Pour l'invite de commande Windows par défaut (cmd)

`.\monenv\Scripts\activate.bat`

* Pour l'invite de commande Windows PowerShell

`.\monenv\Scripts\activate.ps1`

* Pour un invite de commande `bash` et dérivés (Linux, macOS, WSL...)

`source .\monenv\Scripts\activate`

![Activation dans des terminaux](../assets/venv_activate.png)

Tel que montré ci-dessus, que notre environnement virtuel soit activé dans PowerShell ou dans cmd, on voit visuellement la différence avec l'ajout du nom de notre environnement au début de chaque ligne. Cela permet de nous souvenir que nous sommes actuellement "dans" notre environnement virtuel.

<details>
  <summary>【💡 Spoiler】</summary>
  En réalité, l'activation simplement va faire en sorte que, lorsque vous appelez python ou pip dans votre terminal, ce dernier aille utiliser les éxécutables correspondants dans le dossier de votre environnement, plutôt que d'utiliser les exécutables installés sur votre système d'exploitation.
</details>

Pour "sortir" de votre environnement virtuel, vous pouvez bien sûr soit quitter votre invite de commande, soit taper tout simplement ceci :

`deactivate`

![Désactivation dans un terminal](../assets/venv_deactivate.png)

Pareil, vous pouvez constater visuellement la sortie de l'environnement virtuel.

---

## ⚙️ Installer un paquet avec PIP

Lorsque l'on souhaite utiliser des projets tiers qui ne sont pas inclus dans le langage que l'on utilise, il faudra toujours installer ce qu'on appelle des "paquets", et ce à l'aide d'un gestionnaire de paquets.

Dans le cas de Python, l'intégralité des paquets installables sont recensés sur [PyPI](https://pypi.org/), et le gestionnaire de paquets s'appelle PIP.

Contrairement à d'autres langages, un paquet installé en Python sera installé "globalement", c'est à dire que le paquet sera lié à votre installation de Python plutôt qu'à votre projet ou répertoire en cours. Nous verrons plus tard une façon de résoudre ce problème. 

PIP s'appelle dans un terminal comme n'importe quelle autre application, et possède plusieurs commandes pour gérer des paquets tiers. En voici quelques-unes :

* `pip list` : liste tous les paquets installés
* `pip install [paquet]` : installe un paquet
* `pip uninstall [paquet]` : désinstalle un paquet 

Ainsi, si vous souhaitez installer le paquet `Pillow`, il suffira d'écrire `pip install Pillow`

### Le fichier requirements.txt

Pour garder une trace des paquets tiers dont on a besoin dans un projet, on utilise un fichier `requirements.txt` qui peut être écrit et lu par PIP. Combiné aux environnements virtuels, c'est donc quelque chose d'extrêmement pratique. 

Après avoir installé les paquets nécessaires à votre projet au sein d'un environnement virtuel, vous pouvez rapidement exporter la liste de ces paquets puis l'écrire dans le fichier `requirements.txt` en exécutant ceci :

`pip freeze > requirements.txt`

Techniquement, cela va y inscrire la version exacte des paquets installés, afin d'être certain d'assuré la compatibilité avec votre code.

Ensuite, lorsqu'il sera l'occasion de devoir installer tous les paquets nécessaires au projet, la commande `install` pourra lire ce fichier de cette façon :

`pip install -r requirements.txt`

---

## 📚 Bibliothèque tierce - Pillow

Maintenant que nous savons installer des paquets pour utiliser des projets tiers, essayons le projet "Pillow". C'est un dérivé moderne d'une ancienne bibliothèque nommée "PIL" visant à manipuler des images en Python. Aujourd'hui, [Pillow](https://pillow.readthedocs.io/en/stable/index.html) est une bibliothèque de référence lorsqu'il s'agit de lire et manipuler des images.

Pour l'installer, ouvrez un invite de commandes sur votre ordinateur et tapez `pip install Pillow` (il faudra peut-être fermer et rouvrir le notebook après installation).

Commençons l'usage de la bibliothèque par un premier exemple très simple : ouvrir une image, réduire son nombre de couleurs, puis l'afficher.

In [2]:
import os
from PIL import Image

# Construction du chemin vers l'image à l'aide de os.path
image_path = os.path.join("..", "assets", "osaka.jpg")

# Ouverture de l'image en tant que variable "im"
with Image.open(image_path) as im:
    # Réduction des couleurs, puis affichage
    im.quantize(colors=60).show()

Une fois une image ouverte avec Pillow, on a alors une instance de la classe `Image` sur laquelle de nombreuses fonctions sont disponibles. Référez-vous à sa page dans la [documentation](https://pillow.readthedocs.io/en/stable/reference/Image.html) pour une liste exhaustive.

Bien sûr, la méthode `.show()` affichant une image n'est à utiliser qu'en conditions de test comme ici. En temps normal, on voudra plutôt sauvegarder notre image après l'avoir modifiée, et idéalement sous un autre nom pour ne pas écraser l'image originale.

In [4]:
import os
from PIL import Image

path = os.path.join("..", "assets", "osaka.jpg")

with Image.open(path) as im:
    # Réduction de l'image à une taille de 256x256 maximum
    im.thumbnail((256, 256))
    # Récupération du chemin vers le fichier, et de son extension
    filename, ext = os.path.splitext(path)
    # Le nom du nouveau fichier utilise le chemin et l'extension du fichier original
    destination = f"{filename}_thumbnail{ext}"
    # Sauvegarde de l'image au format JPEG
    im.save(destination, "JPEG")

Grâce à une méthode `.thumbnail()`, nous avons rapidement créé une version miniature d'une image existante. C'est quelque chose qui pourra être utile si vous êtes amené à gérer un upload d'image, par exemple.

Une autre façon de manipuler nos images est de dessiner par-dessus. Pour cela, il faudra importer le module `ImageDraw` pour y passer notre image et utiliser [ses nombreuses méthodes](https://pillow.readthedocs.io/en/stable/reference/ImageDraw.html).

Avant de continuer, il faut voir comment fonctionnent le passage de coordonées lorsque l'on appelle des méthodes de ce module. Pour dessiner un rectangle par exemple, au lieu de donner les coordonnées de départ puis une taille en largeur/hauteur, on passe des coordonnées de départ puis des coordonnées d'arrivée, le tout dans un seul tuple.

Pour ce qui est de l'origine des coordonnées, elle est toujours en haut à gauche.

![Explication des coordonnées](../assets/pillow.png)

In [5]:
import os
from PIL import Image, ImageDraw

path = os.path.join("..", "assets", "osaka.jpg")

with Image.open(path) as im:
    # On transmet notre image à une classe permettant d'y dessiner dessus
    draw = ImageDraw.Draw(im)
    # Le premier argument est un tuple indiquant les positions x et y de départ,
    # puis les positions x et y d'arrivée. Pour la couleur de remplissage (fill),
    # il faut l'écrire au format RVB.
    draw.rectangle((280, 150, 700, 220), fill=(29, 49, 72))
    im.show()

Dans le dernier exemple, on a donc dessiné un rectangle à un endroit précis et avec une couleur.

Une autre méthode très utile de `ImageDraw` nous permet d'écrire le texte de notre choix sur une image, en précisant également sa couleur.

In [6]:
import os
from PIL import Image, ImageDraw

path = os.path.join("..", "assets", "osaka.jpg")

color_white = (255, 255, 255)
color_black = (0, 0, 0)

with Image.open(path) as im:
    draw = ImageDraw.Draw(im)
    draw.rectangle((280, 150, 700, 220), fill=color_white)
    # Une position et un texte suffisent à écrire sur l'image
    draw.text((400, 180), "Fukuoka", fill=color_black)
    im.show()

Bien que la fonction d'écriture de texte fonctionne, la police par défaut est assez petite, et sans spécialement être belle non plus.

Pour cela, le module `ImageFont` de Pillow permet d'utiliser un fichier de police d'écriture afin d'écrire du texte en l'utilisant. Le seul petit inconvénient est qu'une police est ouverte à une taille définie : si vous voulez donc utiliser la même police à des tailles différentes, il faudra la préparer à l'avance avec ces tailles.

In [7]:
import os
from PIL import Image, ImageDraw, ImageFont

path = os.path.join("..", "assets", "osaka.jpg")
path_font = os.path.join("..", "assets", "OpenSans-Bold.ttf")

color_white = (255, 255, 255)
color_black = (0, 0, 0)
# On prépare l'usage d'une police, en précisant la taille qui est ici à 42
font_opensans_42pt = ImageFont.truetype(path_font, 42)

with Image.open(path) as im:
    draw = ImageDraw.Draw(im)
    draw.rectangle((280, 150, 700, 220), fill=color_white)
    draw.text((400, 155), "Fukuoka", font=font_opensans_42pt, fill=color_black)
    im.show()

---

# Exercice

Une fois ce cours terminé, vous pourrez réaliser l'exercice du dossier `rugby`.